In [10]:
import pandas as pd
import filtering_with_lineage as fl_lib


In [11]:
swat = pd.read_pickle('pickles/swat_clean.pkl')
swat

,opportunity_id,account,deal_stage,close_date,close_value,sector,revenue,employees
0,0LVWSWEW,Donquadtech,Won,2017-04-13,65.0,technology,1712.68,3194
1,K53BC1T9,Domzoom,Lost,2017-12-15,0.0,entertainment,217.87,551
2,D40N5I9Y,Finhigh,Won,2017-09-28,4739.0,finance,1102.43,1759
3,051QTX8Z,Globex Corporation,Won,2017-04-17,1080.0,technology,1223.72,2497
4,5D2XH04S,Plussunin,Won,2017-03-13,57.0,retail,1419.98,4018
5,V9E582SK,Konex,Won,2017-07-08,469.0,technology,7708.38,13756
6,UK0LEZRJ,Condax,Won,2017-01-21,532.0,medical,4.54,9
7,PFFHM7HR,Opentech,Engaging,NaT,NaN,finance,355.23,853
8,IT10CFQH,Konex,Won,2017-05-02,538.0,technology,7708.38,13756
9,8I5ONXJX,<NA>,Prospecting,NaT,NaN,<NA>,NaN,<NA>


In [12]:
strategic_review = {
    'series_name': 'strategic_review',
    'statements': (
        (('greater than', 'close_value', 1000), 'big deal focus'),
        (('or', (('lesser than', 'close_value', 100), ('is null', 'close_value'))), 'small deal no focus'),
        (('or', (('greater than', 'employees', 3000), ('equal', 'sector', 'technology'))), 'big_tech')),
    'else': 'further review',
    'dtype': 'string'}

swat.case.else_if_all(strategic_review)


In [13]:
# swat.case.lineage is a list of dictionaries, if I put it in the pd.DataFrame I get a table

pd.DataFrame(swat.case.lineage).astype({'logical_id': 'Int64'})

,data_source,logical,logical_id,function_name,input_values,criteria,output,output_name
0,sales with accounts,None,<NA>,greater than,close_value,1000,big deal focus,strategic_review
1,sales with accounts,or,1,lesser than,close_value,100,small deal no focus,strategic_review
2,sales with accounts,or,1,is null,close_value,None,small deal no focus,strategic_review
3,sales with accounts,or,2,greater than,employees,3000,big_tech,strategic_review
4,sales with accounts,or,2,equal,sector,technology,big_tech,strategic_review
5,sales with accounts,None,<NA>,is null,strategic_review,None,further review,strategic_review


In [5]:

# This is a second example of IF...ELSEIF of
# quarter. This is just to demonstrate how filters work
# (there are much easier ways to get a quarter)

month_data = swat['close_date'].dt.month
month_data.name = 'month_of_close_date'

to_quarter = {
    'series_name': 'close_quarter',
    'statements': (
        (('lesser than', month_data, 4), 'Q1'),
        (('lesser than', month_data, 7), 'Q2'),
        (('lesser than', month_data, 10), 'Q3')),
    'else': 'Q4',
    'dtype': 'string'}

swat.case.else_if_all(to_quarter)


In [8]:
# This is how it looks like in lineage attribute (list of dictionaries)
swat.case.lineage[:2]

[{'data_source': 'sales with accounts',
  'logical': None,
  'logical_id': None,
  'function_name': 'greater than',
  'input_values': 'close_value',
  'criteria': '1000',
  'output': 'big deal focus',
  'output_name': 'strategic_review'},
 {'data_source': 'sales with accounts',
  'logical': 'or',
  'logical_id': 1,
  'function_name': 'lesser than',
  'input_values': 'close_value',
  'criteria': '100',
  'output': 'small deal no focus',
  'output_name': 'strategic_review'}]

In [7]:


pd.DataFrame(swat.case.lineage).astype({'logical_id': 'Int64'})

,data_source,logical,logical_id,function_name,input_values,criteria,output,output_name
0,sales with accounts,None,<NA>,greater than,close_value,1000,big deal focus,strategic_review
1,sales with accounts,or,1,lesser than,close_value,100,small deal no focus,strategic_review
2,sales with accounts,or,1,is null,close_value,None,small deal no focus,strategic_review
3,sales with accounts,or,2,greater than,employees,3000,big_tech,strategic_review
4,sales with accounts,or,2,equal,sector,technology,big_tech,strategic_review
5,sales with accounts,None,<NA>,is null,strategic_review,None,further review,strategic_review
6,sales with accounts,None,<NA>,lesser than,month_of_close_date,4,Q1,close_quarter
7,sales with accounts,None,<NA>,lesser than,month_of_close_date,7,Q2,close_quarter
8,sales with accounts,None,<NA>,lesser than,month_of_close_date,10,Q3,close_quarter
9,sales with accounts,None,<NA>,is null,close_quarter,None,Q4,close_quarter


```mermaid
---
config:
    theme: forest
---
flowchart LR
    Start["strategic_review"] --> C0
    C0{"close_value > 1000"} -- Yes --> SR0["big deal focus"]
    C0 -- No --> C1
    C1{"close_value < 100<br/>OR<br/>close_value is null ?"}
    C1 -- Yes --> SR1["small deal no focus"]
    C1 -- No --> C2{"employees > 3000 OR sector = technology?"}
    C2 -- Yes --> SR2["big_tech"]
    C2 -- No --> C3{"strategic_review is null ?"}
    C3 -- Yes --> SR3["further review"]
```

```mermaid
---
config:
    theme: forest
---
flowchart LR

    N1 --> D1
    N1 --> D2
    N1 --> D3
    N2 --> D4
    N3 --> D5
    N4 --> D6
    D1 --> O1
    D2 --> O2
    D3 --> O2
    D4 --> O3
    D5 --> O3
    D6 --> O4

    %% outcomes
    subgraph OT1 [strategic_review]
        O1(big deal focus)
        O2(small deal no focus)
        O3(big tech)
        O4(further review)
    end

    %% decision trees
    D1{> 1000}
    subgraph DT1 [OR]
        D2{<100}
        D3{is Null}
    end

    subgraph DT2 [OR]
        D4{> 3000}
        D5{='big tech'}
    end

    D6{is Null}
    
    %% datasources
    subgraph DB1 [sales with accounts]
        N1(close_value)
        N2(employees)
        N3(sector)
        N4(strategic_review)
    end

```

### This is an example of basic mermaid generic flowchart

```mermaid
---
config:
    theme: forest
---
flowchart LR

    N1 --> D1
    D1 -- True --> O1
    D1 -- False --> O2
    N2 --> D2
    D2 -- True --> O3
    D2 -- False --> O4

    %% datasource
    subgraph MT [My Table]
        N1(Column1)
        N2(Column2)
    end
    %% decision
    D1{Rule1}
    D2{Rule2}

    %% outcomes
    O1[(Outcome1)]
    O2[[Outcome2]]
    O3>Outcome3]
    O4{{Outcome4}}
```